In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
from xgboost import XGBRegressor

In [4]:
df=pd.read_csv('data/stud.csv')
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [5]:
x=df.drop(columns=['reading_score'],axis=1)

In [6]:
y=df['reading_score']

In [7]:
num_features=x.select_dtypes(exclude='object').columns
cat_features=x.select_dtypes(include='object').columns
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transform=StandardScaler()
oh_transform=OneHotEncoder()

preprocessor=ColumnTransformer(
    [
    ('OneHotEncoder',oh_transform,cat_features),
    ('StandardScaler',numeric_transform,num_features),
    ]
)

In [8]:
X=preprocessor.fit_transform(x)

In [9]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
x_train.shape,x_test.shape

((800, 19), (200, 19))

In [14]:
def evaluate_model(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mean_squared_error(true,predicted))
    r2_squared=r2_score(true,predicted)
    return mae,rmse,r2_squared

In [15]:
models={
    'Linear Regression': LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(),
    "AdaBoost Regressor": AdaBoostRegressor(),
    "CatBoosting Regressor": CatBoostRegressor(verbose=False)
}
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(x_train,y_train)

    y_train_pred=model.predict(x_train)
    y_test_pred=model.predict(x_test)

    model_train_mae,model_train_rmse,model_train_r2=evaluate_model(y_train,y_train_pred)
    model_test_mae,model_test_rmse,model_test_r2=evaluate_model(y_test,y_test_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print("Model Performance for Training set")
    print("- Root Mean Squared Error:{:.4f}".format(model_train_rmse))
    print("- Mean absolute Error:{:.4f}".format(model_train_mae))
    print("- R2 Score:.{:.4f}".format(model_train_r2))

    print("-------------------------------------------")
    print("Model Performance for Testing set")
    print("- Root Mean Squared Error:{:.4f}".format(model_test_rmse))
    print("- Mean absolute Error:{:.4f}".format(model_test_mae))
    print("- R2 Score:.{:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)

    print("="*45)
    print("\n")

Linear Regression
Model Performance for Training set
- Root Mean Squared Error:3.8718
- Mean absolute Error:3.1220
- R2 Score:.0.9282
-------------------------------------------
Model Performance for Testing set
- Root Mean Squared Error:4.3088
- Mean absolute Error:3.4908
- R2 Score:.0.9180


Lasso
Model Performance for Training set
- Root Mean Squared Error:4.2215
- Mean absolute Error:3.4220
- R2 Score:.0.9147
-------------------------------------------
Model Performance for Testing set
- Root Mean Squared Error:4.6350
- Mean absolute Error:3.6941
- R2 Score:.0.9051


Ridge
Model Performance for Training set
- Root Mean Squared Error:3.8720
- Mean absolute Error:3.1221
- R2 Score:.0.9282
-------------------------------------------
Model Performance for Testing set
- Root Mean Squared Error:4.3028
- Mean absolute Error:3.4861
- R2 Score:.0.9182


K-Neighbors Regressor
Model Performance for Training set
- Root Mean Squared Error:4.8003
- Mean absolute Error:3.7423
- R2 Score:.0.8897
-

In [18]:
pd.DataFrame(list(zip(model_list,r2_list)),columns=['Model name','R2_Score']).sort_values(by=["R2_Score"],ascending=False)

,Model name,R2_Score
2,Ridge,0.918184
0,Linear Regression,0.917955
5,Random Forest Regressor,0.912437
8,CatBoosting Regressor,0.907223
1,Lasso,0.905059
7,AdaBoost Regressor,0.903998
6,XGBRegressor,0.899641
3,K-Neighbors Regressor,0.837096
4,Decision Tree,0.832357
